In [1]:
import pandas as pd
gender_age_train = pd.read_csv('gender_age_train.csv',dtype={'device_id':'string','gender':'string','age':'string','group':'string'})
gender_age_train['flag'] = 'train'
gender_age_test = pd.read_csv('gender_age_test.csv',dtype={'device_id':'string'})
gender_age_test['flag'] = 'test'
gender = gender_age_train.append(gender_age_test)
events = pd.read_csv('events.csv',dtype={'event_id':'string','device_id':'string','timestamp':'string','longitude':'string','latitude':'string'})


In [2]:
## 手机使用数量
phone_brand_device_model = pd.read_csv('phone_brand_device_model.csv',dtype={'device_id':'string','phone_brand':'string','device_model':'string'})
device = phone_brand_device_model.groupby('device_id').agg({'phone_brand':'count','device_model':'count'})
device['device_id'] = device.index
device.index=range(len(device))
device.columns=['device_model_num','phone_brand_num','device_id']
phone_device={}
for x in phone_brand_device_model.values:
    if phone_device.get(x[0]) is None:
       phone_device[x[0]] =  {'phone_brand':x[1],'device_model':x[2]}
    else:
      phone_device[x[0]] =  {'phone_brand':x[1],'device_model':x[2]}
device['phone_brand']  = device['device_id'].map(lambda x:phone_device[x]['phone_brand'])
device['device_model'] = device['device_id'].map(lambda x:phone_device[x]['device_model'])
data = pd.merge(gender,device,on=['device_id'],how='left')


In [3]:
##手机出现频率
phone_brand = data.groupby('phone_brand').agg({'phone_brand':'count'})
device_model = data.groupby('device_model').agg({'device_model':'count'})
device_model.columns=['device_model_total']
device_model['device_model'] = device_model.index
phone_brand.columns=['phone_brand_total']
phone_brand['phone_brand'] = phone_brand.index
phone_brand['phone_brand_total'] = phone_brand['phone_brand_total']/1.0/phone_brand['phone_brand_total'].sum()
device_model['device_model_total'] = device_model['device_model_total']/1.0/device_model['device_model_total'].sum()
data = pd.merge(data,phone_brand,on=['phone_brand'],how='left')
data = pd.merge(data,device_model,on=['device_model'],how='left')

In [4]:
##设备发生的事件数量
events_num = events.groupby('device_id').agg({'event_id':'count'})
events_num.columns=['events_num']
events_num['device_id'] = events_num.index
data = pd.merge(data,events_num,on=['device_id'],how='left')
##事件缺少值填充，以设备和手机平均值填充



In [5]:
##事件发生的最大时间与最小时间
events_max = events.groupby(['device_id']).agg({'timestamp':'max'})
events_min = events.groupby(['device_id']).agg({'timestamp':'min'})
events_max['device_id'] = events_max.index
events_min['device_id'] = events_min.index
import datetime as dt

events_min.columns=['timestamp_min','device_id']
events_max.columns=['timestamp_max','device_id']
events_date = pd.merge(events_min,events_max,on=['device_id'])
events_date['times'] = map(lambda x,y:
                           (dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') -
                            dt.datetime.strptime(y,'%Y-%m-%d %H:%M:%S')).total_seconds(),
                            events_date['timestamp_max'],events_date['timestamp_min'])
data = pd.merge(data,events_date[['device_id','times']],on=['device_id'],how='left')
#data = data.fillna({'times':data['times'].mean()})

In [6]:
##每天操作手机的评率
events['day'] = events['timestamp'].map(lambda x:dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S').day)
events_day = events.groupby(['device_id','day']).agg({'day':'count'})
#events_day = events_day.to_dict('records')
events_day = map(lambda x:x[0]+(x[1],),events_day.to_dict()['day'].items())
events_day = pd.DataFrame(events_day,columns=['device_id','day','events_day_num']).groupby('device_id').agg({'events_day_num':'mean'})
events_day['device_id'] = events_day.index
data = pd.merge(data,events_day,on=['device_id'],how='left')

In [7]:
##操作手机频率
events['timestamp'] = events['timestamp'].map(lambda x:dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
events['t_7-12']  = events['timestamp'].map(lambda x: 1 if x.hour>=7 and x.hour<=12 else 0)
events['t_12-18'] = events['timestamp'].map(lambda x: 1 if x.hour>12 and x.hour<=18 else 0)
events['t_18-00'] = events['timestamp'].map(lambda x: 1 if x.hour>18  else 0)
events['t_00-7']  = events['timestamp'].map(lambda x: 1 if x.hour>=0 and x.hour<7 else 0)
events_time = events.groupby('device_id').agg({'t_7-12':'sum','t_12-18':'sum','t_18-00':'sum','t_00-7':'sum'})
events_time['t'] = events_time['t_7-12']+events_time['t_12-18']+events_time['t_18-00']+events_time['t_00-7']
events_time['t_7-12']  = events_time['t_7-12']/events_time['t']
events_time['t_12-18'] =  events_time['t_12-18']/events_time['t']
events_time['t_18-00'] =  events_time['t_18-00']/events_time['t']
events_time['t_00-7']  =  events_time['t_00-7']/events_time['t']
events_time['device_id']=events_time.index
data = pd.merge(data,events_time[['device_id','t_7-12','t_18-00','t_00-7','t_12-18']],on=['device_id'],how='left')


In [8]:
##活动范围
events['longitude'] = events['longitude'].astype('float')
events['latitude'] = events['latitude'].astype('float')
events = events[(events['longitude']!=0) & (events['latitude']!=0)]
events_max  = events[(events['longitude']!=0) & (events['latitude']!=0)].groupby(['device_id']).agg({'longitude':'max','latitude':'max'})
events_max['device_id'] = events_max.index
events_max.columns =['latitude_max','longitude_max','device_id']
events_min  = events[(events['longitude']!=0) & (events['latitude']!=0)].groupby(['device_id']).agg({'longitude':'min','latitude':'min'})
events_min['device_id'] = events_min.index
events_min.columns =['latitude_min','longitude_min','device_id']


events_mean = events[(events['longitude']!=0) & (events['latitude']!=0)].groupby(['device_id']).agg({'longitude':'mean','latitude':'mean'})
events_mean['device_id'] = events_mean.index
events_mean.columns =['latitude_mean','longitude_mean','device_id']

events_lat_lng = pd.merge(events_max,events_min,on='device_id')
events_lat_lng = pd.merge(events_lat_lng,events_mean,on='device_id')
data = pd.merge(data,events_lat_lng,on=['device_id'],how='left')

##活动面积
data['squre'] = (data['longitude_max'] - data['longitude_min'])*(data['latitude_max'] - data['latitude_min'])

In [9]:
from sklearn.feature_extraction import DictVectorizer


def getcategory():
    with open('category.csv') as f:
        data =  f.readlines()
        data = data[1:]
    
    def getDict(line):
        temp = line.strip().replace('"','').split(',')
        key = temp[0]
        value = temp[1:]
        lst = {}
        for s in value:
            s_lower = s.lower()
            if lst.get(s) is None:
                lst[s] = 1
            else:
                lst[s] = lst.get(s) + 1

        return  key, lst
    
    Dict = map(lambda x: getDict(x),data)

    vec = DictVectorizer()

    Features = vec.fit_transform(map(lambda x:x[1],Dict)).toarray()
    return map(lambda x,y:[x[0]] + y.tolist(),Dict,Features)

category = getcategory()
colname = ['x_%d'%i for i in range(len(category[0][1:]))]
category = pd.DataFrame(category,columns=['device_id']+colname)

In [10]:
modeldata = pd.merge(data,category,on=['device_id'],how='left')
#modeldata = modeldata.fillna(dict(map(lambda x:(x,modeldata[x].mean()),colname)))


In [11]:
#del events_lat_lng,events_max,events_mean,events_min,events_num,events_time,gender,gender_age_test,gender_age_train
#del phone_brand,phone_brand_device_model,phone_device
modeldata.to_csv('modeldata_1.csv',index=False)

In [1]:
import pandas as pd
modeldata = pd.read_csv('modeldata_1.csv')

/Users/lywen/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def genfeatures(data,featureslabel):
    fdict = data[featureslabel].to_dict('records')
    vec = DictVectorizer()
    Features = vec.fit_transform(fdict).toarray()
    return Features
import xgboost
def modeltrain(trainX,trainY,testX):
    
    fit =xgboost.sklearn.XGBRegressor(learning_rate =0.3,
                                         n_estimators=30,
                                         max_depth=6,
                                         min_child_weight=4,
                                         gamma=1,
                                         subsample=0.7,
                                         colsample_bytree=0.8,
                                         reg_alpha =0.10,
                                         reg_lambda=1.0,
                                         objective= 'reg:linear',nthread=-1, scale_pos_weight=1,seed=27)
    model = fit.fit(trainX,trainY,eval_set=[(trainX,trainY)],eval_metric='rmse')
    testY = model.predict(testX)
    return testY
def datafill(data,label,featureslabel):
    Features = genfeatures(data,featureslabel)
    testindex = data[data[label].isnull()].index
    trainindex = data[data[label].notnull()].index
    trainX = Features[trainindex]
    testX = Features[testindex]
    trainY = data[label].values[trainindex]
    testY  = modeltrain(trainX,trainY,testX)
    data[label].values[testindex] = testY
from sklearn.feature_extraction import DictVectorizer
    

In [ ]:
label = 'events_num'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:200.374802
[1]	validation_0-rmse:198.390976
[2]	validation_0-rmse:197.405258
[3]	validation_0-rmse:196.712463
[4]	validation_0-rmse:196.401855


In [9]:
label = 'times'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total','events_num']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:329390.218750
[1]	validation_0-rmse:251474.328125
[2]	validation_0-rmse:219589.343750
[3]	validation_0-rmse:183457.984375
[4]	validation_0-rmse:162697.656250
[5]	validation_0-rmse:151339.906250
[6]	validation_0-rmse:145404.359375
[7]	validation_0-rmse:142288.812500
[8]	validation_0-rmse:140701.343750
[9]	validation_0-rmse:139887.687500
[10]	validation_0-rmse:139378.906250
[11]	validation_0-rmse:139128.187500
[12]	validation_0-rmse:138944.578125
[13]	validation_0-rmse:138832.750000
[14]	validation_0-rmse:138718.531250
[15]	validation_0-rmse:138642.546875
[16]	validation_0-rmse:138560.046875
[17]	validation_0-rmse:138515.250000
[18]	validation_0-rmse:138486.187500
[19]	validation_0-rmse:138446.593750
[20]	validation_0-rmse:138428.343750
[21]	validation_0-rmse:138398.265625
[22]	validation_0-rmse:138356.703125
[23]	validation_0-rmse:138300.828125
[24]	validation_0-rmse:138262.656250
[25]	validation_0-rmse:138251.703125
[26]	validation_0-rmse:138227.843750
[27]	valida

In [13]:
label = 'events_day_num'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total','events_num','times']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:23.650448
[1]	validation_0-rmse:20.927357
[2]	validation_0-rmse:20.208324
[3]	validation_0-rmse:18.538755
[4]	validation_0-rmse:17.659948
[5]	validation_0-rmse:16.627016
[6]	validation_0-rmse:16.289856
[7]	validation_0-rmse:15.185940
[8]	validation_0-rmse:14.318143
[9]	validation_0-rmse:13.719980
[10]	validation_0-rmse:13.363334
[11]	validation_0-rmse:13.011360
[12]	validation_0-rmse:12.886346
[13]	validation_0-rmse:12.581799
[14]	validation_0-rmse:12.259175
[15]	validation_0-rmse:12.217054
[16]	validation_0-rmse:11.834588
[17]	validation_0-rmse:11.833856
[18]	validation_0-rmse:11.595669
[19]	validation_0-rmse:11.310611
[20]	validation_0-rmse:11.337651
[21]	validation_0-rmse:11.150907
[22]	validation_0-rmse:11.026711
[23]	validation_0-rmse:10.646741
[24]	validation_0-rmse:10.300741
[25]	validation_0-rmse:10.305758
[26]	validation_0-rmse:10.191594
[27]	validation_0-rmse:10.135381
[28]	validation_0-rmse:10.062782
[29]	validation_0-rmse:10.083966


In [14]:
label = 't_7-12'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total','events_num','times','events_day_num']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:0.313685
[1]	validation_0-rmse:0.298751
[2]	validation_0-rmse:0.290891
[3]	validation_0-rmse:0.287068
[4]	validation_0-rmse:0.285107
[5]	validation_0-rmse:0.284166
[6]	validation_0-rmse:0.283467
[7]	validation_0-rmse:0.283170
[8]	validation_0-rmse:0.283072
[9]	validation_0-rmse:0.282990
[10]	validation_0-rmse:0.282734
[11]	validation_0-rmse:0.282723
[12]	validation_0-rmse:0.282718
[13]	validation_0-rmse:0.282705
[14]	validation_0-rmse:0.282654
[15]	validation_0-rmse:0.282655
[16]	validation_0-rmse:0.282607
[17]	validation_0-rmse:0.282606
[18]	validation_0-rmse:0.282604
[19]	validation_0-rmse:0.282599
[20]	validation_0-rmse:0.282604
[21]	validation_0-rmse:0.282603
[22]	validation_0-rmse:0.282606
[23]	validation_0-rmse:0.282530
[24]	validation_0-rmse:0.282530
[25]	validation_0-rmse:0.282506
[26]	validation_0-rmse:0.282507
[27]	validation_0-rmse:0.282507
[28]	validation_0-rmse:0.282465
[29]	validation_0-rmse:0.282442


In [15]:
label = 't_18-00'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total','events_num','times','events_day_num','t_7-12']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:0.323887
[1]	validation_0-rmse:0.288925
[2]	validation_0-rmse:0.269928
[3]	validation_0-rmse:0.259888
[4]	validation_0-rmse:0.254671
[5]	validation_0-rmse:0.251932
[6]	validation_0-rmse:0.251082
[7]	validation_0-rmse:0.250208
[8]	validation_0-rmse:0.250005
[9]	validation_0-rmse:0.249288
[10]	validation_0-rmse:0.248942
[11]	validation_0-rmse:0.248873
[12]	validation_0-rmse:0.248629
[13]	validation_0-rmse:0.248570
[14]	validation_0-rmse:0.248564
[15]	validation_0-rmse:0.248562
[16]	validation_0-rmse:0.248503
[17]	validation_0-rmse:0.248509
[18]	validation_0-rmse:0.248507
[19]	validation_0-rmse:0.248506
[20]	validation_0-rmse:0.248509
[21]	validation_0-rmse:0.248460
[22]	validation_0-rmse:0.248459
[23]	validation_0-rmse:0.248460
[24]	validation_0-rmse:0.248458
[25]	validation_0-rmse:0.248380
[26]	validation_0-rmse:0.248342
[27]	validation_0-rmse:0.248336
[28]	validation_0-rmse:0.248264
[29]	validation_0-rmse:0.248192


In [16]:
label = 't_00-7'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total','events_num','times','events_day_num','t_7-12','t_18-00']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:0.334563
[1]	validation_0-rmse:0.280329
[2]	validation_0-rmse:0.241111
[3]	validation_0-rmse:0.221373
[4]	validation_0-rmse:0.211774
[5]	validation_0-rmse:0.202459
[6]	validation_0-rmse:0.197488
[7]	validation_0-rmse:0.194856
[8]	validation_0-rmse:0.193489
[9]	validation_0-rmse:0.192855
[10]	validation_0-rmse:0.192661
[11]	validation_0-rmse:0.192132
[12]	validation_0-rmse:0.191968
[13]	validation_0-rmse:0.191836
[14]	validation_0-rmse:0.191597
[15]	validation_0-rmse:0.191592
[16]	validation_0-rmse:0.191445
[17]	validation_0-rmse:0.191448
[18]	validation_0-rmse:0.191448
[19]	validation_0-rmse:0.191445
[20]	validation_0-rmse:0.191316
[21]	validation_0-rmse:0.191315
[22]	validation_0-rmse:0.191254
[23]	validation_0-rmse:0.191140
[24]	validation_0-rmse:0.191064
[25]	validation_0-rmse:0.191067
[26]	validation_0-rmse:0.191067
[27]	validation_0-rmse:0.191064
[28]	validation_0-rmse:0.191005
[29]	validation_0-rmse:0.190898


In [17]:
label = 't_12-18'
featureslabel = ['device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total','events_num','times','events_day_num','t_7-12','t_18-00','t_00-7']
datafill(modeldata,label,featureslabel)

[0]	validation_0-rmse:0.305021
[1]	validation_0-rmse:0.270741
[2]	validation_0-rmse:0.251793
[3]	validation_0-rmse:0.185577
[4]	validation_0-rmse:0.138052
[5]	validation_0-rmse:0.134346
[6]	validation_0-rmse:0.132385
[7]	validation_0-rmse:0.131272
[8]	validation_0-rmse:0.129039
[9]	validation_0-rmse:0.097217
[10]	validation_0-rmse:0.085133
[11]	validation_0-rmse:0.085060
[12]	validation_0-rmse:0.071415
[13]	validation_0-rmse:0.056958
[14]	validation_0-rmse:0.047534
[15]	validation_0-rmse:0.043980
[16]	validation_0-rmse:0.038491
[17]	validation_0-rmse:0.037352
[18]	validation_0-rmse:0.036104
[19]	validation_0-rmse:0.034135
[20]	validation_0-rmse:0.032163
[21]	validation_0-rmse:0.031419
[22]	validation_0-rmse:0.030905
[23]	validation_0-rmse:0.030905
[24]	validation_0-rmse:0.030905
[25]	validation_0-rmse:0.030466
[26]	validation_0-rmse:0.029904
[27]	validation_0-rmse:0.029904
[28]	validation_0-rmse:0.029681
[29]	validation_0-rmse:0.029681


In [18]:
modeldata['group'] = modeldata['group'].fillna('N')

groupDict = dict(zip(pd.unique(modeldata['group']),range(len(pd.unique(modeldata['group'])))))

modeldata['label'] = modeldata['group'].map(lambda x:groupDict[x])



In [20]:
from sklearn.cross_validation   import train_test_split
from sklearn.feature_extraction import DictVectorizer

#ModelData['target']=ModelData['target'].astype('int')
vec = DictVectorizer()
featureLables=featureslabel+[label]
target = 'label'
FeaturesDict = modeldata[featureLables].to_dict('record')
Features = vec.fit_transform(FeaturesDict).toarray()

LabelsDict = modeldata[target].to_frame().to_dict('record')

Labels = vec.fit_transform(LabelsDict).toarray().ravel()


In [21]:
train_index = modeldata[modeldata['flag']=='train'].index
                        
test_index = modeldata[modeldata['flag']=='test'].index
trainX,trainY = Features[train_index],Labels[train_index]


In [26]:

import xgboost
fit = xgboost.sklearn.XGBClassifier(learning_rate =0.3,
 n_estimators=50,
 max_depth=8,
 min_child_weight=4,
 gamma=1,
 subsample=0.7,
 colsample_bytree=0.8,
 reg_alpha = .10,
 reg_lambda = 1.0,
 objective= 'multi:softmax',nthread=-1, scale_pos_weight=1,seed=27)
model = fit.fit(trainX,trainY,eval_set=[(trainX,trainY)],eval_metric = 'mlogloss')

[0]	validation_0-mlogloss:2.446971
[1]	validation_0-mlogloss:2.419878
[2]	validation_0-mlogloss:2.399255
[3]	validation_0-mlogloss:2.383800
[4]	validation_0-mlogloss:2.369845
[5]	validation_0-mlogloss:2.359297
[6]	validation_0-mlogloss:2.350273
[7]	validation_0-mlogloss:2.342395
[8]	validation_0-mlogloss:2.336531
[9]	validation_0-mlogloss:2.331993
[10]	validation_0-mlogloss:2.327736
[11]	validation_0-mlogloss:2.323910
[12]	validation_0-mlogloss:2.320038
[13]	validation_0-mlogloss:2.316168
[14]	validation_0-mlogloss:2.313592
[15]	validation_0-mlogloss:2.310502
[16]	validation_0-mlogloss:2.308142
[17]	validation_0-mlogloss:2.305079
[18]	validation_0-mlogloss:2.303158
[19]	validation_0-mlogloss:2.301074
[20]	validation_0-mlogloss:2.299217
[21]	validation_0-mlogloss:2.297232
[22]	validation_0-mlogloss:2.294759
[23]	validation_0-mlogloss:2.292868
[24]	validation_0-mlogloss:2.290672
[25]	validation_0-mlogloss:2.289003
[26]	validation_0-mlogloss:2.287350
[27]	validation_0-mlogloss:2.285265
[2

In [27]:
A = model.predict_proba(Features[test_index])
def getorder(x):
    colname =['F23-', 'F24-26','F27-28','F29-32', 'F33-42', 'F43+', 'M22-', 'M23-26', 'M27-28', 'M29-31', 'M32-38', 'M39+']
    tmp = []
    for i in colname:
        tmp.append(x[groupDict[i]])
    return tmp

testdevice_id = modeldata['device_id'].values[test_index]
testlabel = A#[test_index]



In [28]:
result = map(lambda x,y:[x] + getorder(y),testdevice_id,testlabel)


columns = 'device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+'.split(',')

result1 = (pd.DataFrame(result,columns=columns))

result1.to_csv('result10_1.csv',index=False)

In [29]:
result1.head(10)

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
0,1002079943728939269,0.032392,0.059800,0.026908,0.063340,0.152446,0.124797,0.064577,0.081673,0.069360,0.092282,0.101328,0.131098
1,-1547860181818787117,0.050137,0.058300,0.031023,0.064537,0.059070,0.131193,0.071428,0.215249,0.021835,0.077307,0.135966,0.083956
2,7374582448058474277,0.057859,0.046878,0.033557,0.102589,0.065396,0.040105,0.061412,0.112013,0.060381,0.103390,0.160808,0.155610
3,-6220210354783429585,0.018706,0.056197,0.020075,0.046863,0.052220,0.035116,0.082302,0.087827,0.101966,0.236132,0.109809,0.152787
4,-5893464122623104785,0.039855,0.078900,0.043878,0.065407,0.041628,0.041089,0.077639,0.168531,0.105186,0.103538,0.141688,0.092661
5,-7560708697029818408,0.039855,0.078900,0.043878,0.065407,0.041628,0.041089,0.077639,0.168531,0.105186,0.103538,0.141688,0.092661
6,289797889702373958,0.064927,0.061382,0.039320,0.069736,0.079049,0.057036,0.099992,0.121515,0.075719,0.096551,0.124467,0.110306
7,-402874006399730161,0.064927,0.061382,0.039320,0.069736,0.079049,0.057036,0.099992,0.121515,0.075719,0.096551,0.124467,0.110306
8,5751283639860028129,0.051274,0.063776,0.046839,0.071137,0.083167,0.072356,0.069892,0.127260,0.078179,0.097507,0.139504,0.099110
9,-848943298935149395,0.051274,0.063776,0.046839,0.071137,0.083167,0.072356,0.069892,0.127260,0.078179,0.097507,0.139504,0.099110


In [25]:
result1

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
0,1002079943728939269,0.030776,0.049333,0.039987,0.042431,0.184374,0.082934,0.055414,0.086482,0.046595,0.101511,0.132286,0.147878
1,-1547860181818787117,0.065500,0.079308,0.029254,0.065167,0.048042,0.129584,0.091131,0.138057,0.038044,0.090531,0.124782,0.100601
2,7374582448058474277,0.049477,0.032458,0.035796,0.056018,0.062107,0.075189,0.062157,0.071458,0.079090,0.113622,0.128281,0.234348
3,-6220210354783429585,0.034868,0.047140,0.023215,0.052586,0.064529,0.051220,0.048705,0.113405,0.152682,0.128715,0.106450,0.176483
4,-5893464122623104785,0.042288,0.080617,0.043753,0.064513,0.041443,0.039707,0.078099,0.163251,0.108977,0.104444,0.140529,0.092379
5,-7560708697029818408,0.042288,0.080617,0.043753,0.064513,0.041443,0.039707,0.078099,0.163251,0.108977,0.104444,0.140529,0.092379
6,289797889702373958,0.064758,0.059946,0.037557,0.068391,0.081195,0.058596,0.098374,0.118927,0.075585,0.097068,0.125940,0.113663
7,-402874006399730161,0.064758,0.059946,0.037557,0.068391,0.081195,0.058596,0.098374,0.118927,0.075585,0.097068,0.125940,0.113663
8,5751283639860028129,0.048954,0.067026,0.045739,0.070600,0.080722,0.074243,0.071882,0.131878,0.076072,0.097069,0.141021,0.094793
9,-848943298935149395,0.048954,0.067026,0.045739,0.070600,0.080722,0.074243,0.071882,0.131878,0.076072,0.097069,0.141021,0.094793


In [42]:
print modeldata.columns.tolist()

['age', 'device_id', 'flag', 'gender', 'group', 'device_model_num', 'phone_brand_num', 'phone_brand', 'device_model', 'phone_brand_total', 'device_model_total', 'events_num', 'times', 'events_day_num', 't_7-12', 't_18-00', 't_00-7', 't_12-18', 'latitude_max', 'longitude_max', 'latitude_min', 'longitude_min', 'latitude_mean', 'longitude_mean', 'squre', 'x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78', 'x_79', 'x_80', 'x_81', 